# Time Series Prediction with Ember ML

This notebook demonstrates how to build and train a recurrent neural network (RNN) for time series prediction using the Ember ML framework. We will use a synthetic sine wave dataset as an example and showcase Ember ML's backend-agnostic capabilities.

In [ ]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt

# Import Ember ML components
from ember_ml.ops import set_backend
from ember_ml.nn import tensor
from ember_ml import ops
from ember_ml.nn.modules.rnn import GRU # Using GRU as an example RNN
from ember_ml.training import Adam, MSELoss # Using Adam optimizer and MSE loss

# Set a backend (choose 'numpy', 'torch', or 'mlx')
# You can change this to see how the code runs on different backends
set_backend('numpy')
print(f"Using backend: {ops.get_backend()}")

## 1. Generate Synthetic Time Series Data

We will generate a simple sine wave with some noise to simulate time series data.

In [ ]:
def generate_sine_wave_data(num_samples, seq_length, freq=0.05, noise_level=0.1):
    """Generates synthetic sine wave data."""
    t = tensor.linspace(0, seq_length * freq * 2 * ops.pi, num_samples * seq_length)
    sine_wave = ops.sin(t).reshape(num_samples, seq_length, 1)
    noise = np.random.randn(*sine_wave.shape) * noise_level
    noisy_sine_wave = sine_wave + noise
    
    # Create input sequences (X) and target sequences (y)
    # Predict the next step in the sequence
    X = noisy_sine_wave[:, :-1, :]
    y = noisy_sine_wave[:, 1:, :]
    
    return tensor.convert_to_tensor(X, dtype=tensor.float32), tensor.convert_to_tensor(y, dtype=tensor.float32)

# Generate data
num_samples = 1000
seq_length = 50
X_train, y_train = generate_sine_wave_data(num_samples, seq_length)

print(f"Training data shape (X): {tensor.shape(X_train)}")
print(f"Training data shape (y): {tensor.shape(y_train)}")

## 2. Define the RNN Model

We will use a simple GRU model for this prediction task.

In [ ]:
# Define model parameters
input_size = tensor.shape(X_train)[-1]
hidden_size = 32
output_size = tensor.shape(y_train)[-1]

# Create the GRU model
# We set return_sequences=True to predict each step in the sequence
model = GRU(input_size=input_size, hidden_size=hidden_size, batch_first=True, return_sequences=True)

print("Model Architecture:")
print(model)

## 3. Set up Training

We'll use the Adam optimizer and Mean Squared Error (MSE) loss.

In [ ]:
# Create optimizer and loss function
learning_rate = 0.01
optimizer = Adam(learning_rate=learning_rate)
loss_fn = MSELoss()

# Get trainable variables from the model
trainable_variables = model.trainable_variables

print(f"Number of trainable variables: {len(trainable_variables)}")

## 4. Train the Model

We will train the model using a manual training loop, computing gradients and updating weights using the optimizer. Since Ember ML provides `ops.gradients` and optimizer's `apply_gradients`, we can construct a standard training loop.

In [ ]:
epochs = 50
batch_size = 32

print("Starting training...")

for epoch in range(epochs):
    total_loss = 0
    num_batches = 0
    
    # Iterate over batches
    for i in range(0, tensor.shape(X_train)[0], batch_size):
        batch_X = X_train[i:i+batch_size]
        batch_y = y_train[i:i+batch_size]
        
        # Forward pass
        with ops.GradientTape() as tape:
             predictions = model(batch_X)
             loss = loss_fn(batch_y, predictions)
        
        # Compute gradients
        gradients = tape.gradient(loss, trainable_variables)
        
        # Apply gradients
        optimizer.apply_gradients(zip(gradients, trainable_variables))
        
        total_loss = ops.add(total_loss, loss)
        num_batches += 1
        
    avg_loss = ops.divide(total_loss, tensor.convert_to_tensor(num_batches, dtype=tensor.float32))
    print(f"Epoch {epoch+1}/{epochs}, Loss: {tensor.to_numpy(avg_loss):.4f}")

print("Training finished.")

## 5. Make Predictions and Visualize

Let's use the trained model to make predictions on a sample sequence and visualize the result.

In [ ]:
# Take a sample sequence from the training data
sample_input_sequence = X_train[0:1]
sample_true_sequence = y_train[0:1]

# Make a prediction
# Set model to evaluation mode (important for layers like Dropout if used)
model.eval()
predicted_sequence = model(sample_input_sequence)

# Convert to NumPy for plotting
sample_input_np = tensor.to_numpy(sample_input_sequence).squeeze()
sample_true_np = tensor.to_numpy(sample_true_sequence).squeeze()
predicted_np = tensor.to_numpy(predicted_sequence).squeeze()

# Plot the results
plt.figure(figsize=(12, 6))
plt.plot(np.arange(tensor.shape(sample_input_sequence)[1]), sample_input_np, label='Input Sequence')
plt.plot(np.arange(tensor.shape(sample_input_sequence)[1], tensor.shape(sample_input_sequence)[1] + tensor.shape(sample_true_sequence)[1]), sample_true_np, label='True Future Sequence', color='green')
plt.plot(np.arange(tensor.shape(sample_input_sequence)[1], tensor.shape(sample_input_sequence)[1] + tensor.shape(predicted_sequence)[1]), predicted_np, label='Predicted Future Sequence', color='red', linestyle='--')

plt.title('Time Series Prediction Example')
plt.xlabel('Time Step')
plt.ylabel('Value')
plt.legend()
plt.grid(True)
plt.show()

## Conclusion

This notebook demonstrated how to use Ember ML's RNN modules, optimizers, and loss functions for a simple time series prediction task. By leveraging the backend-agnostic API, the same code can be executed on different computational backends. You can extend this example to more complex datasets and model architectures available in Ember ML.